In [0]:
# add widzgets
dbutils.widgets.text("p_data_source","")
v_data_source = dbutils.widgets.get("p_data_source")
print(v_data_source)

In [0]:
%run "../Includes/Configuration"

In [0]:
%run "../Includes/Common_Functions"

In [0]:
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,DoubleType,DateType

In [0]:
driver_schema = StructType([
    StructField('driverId', IntegerType(),True),
    StructField('driverRef', StringType(),True),
    StructField('number', IntegerType(),True),
    StructField('code', StringType(),True),
    StructField('name', StructType([StructField('forename', StringType(), True), StructField('surname', StringType(), True)
    ]), True),
    StructField('dob', DateType(),True),
    StructField('nationality', StringType(), True),
    StructField('url', StringType(),True)
])
df = spark.read.json('abfss://udayadbcontainerv1@udayadbsav1.dfs.core.windows.net/RAW-V1/drivers.json', schema=driver_schema)

In [0]:
df.display()

In [0]:
from pyspark.sql.functions import concat,lit,col

In [0]:
df = add_ingestion_date(df)

In [0]:
df = df.withColumnRenamed('driverId','driver_Id')\
       .withColumnRenamed('driverRef', 'driver_Ref')\
       .withColumn("Data_Source",lit(v_data_source))\
       .withColumn('name',concat(col('name.forename'),lit(" "),col('name.surname')))\
       .drop('url')

In [0]:
df.display()

In [0]:
df.write.mode('overwrite').parquet(f"{Processed_path}/Drivers")

In [0]:
dbutils.notebook.exit("Sucessed")